In [1]:
# import things
%matplotlib notebook
import tensorflow as tf
from tensorflow import keras
from mpl_toolkits import mplot3d
import numpy as np
from numpy import matlib
from scipy import signal
from scipy.spatial import distance
from scipy.stats import norm
from scipy.stats import percentileofscore
import random
import matplotlib.pyplot as plt
import os

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import pairwise_distances
from scipy.optimize import least_squares

from dPCA import dPCA

In [2]:
# import desired emg or velocity trajectories
predict_emg = 1
do_emg_norm = 1
go_cue_idx = 20
hold_idx = 10

%run ./get_stim_dynamics_inputs.ipynb
x_train,y_train,x_test,y_test = get_input_data(predict_emg,do_emg_norm,go_cue_idx,hold_idx)
plt.plot(y_train[1,:,:]);

# params
%run Param_classes.ipynb
args = InitialParams()   
stim_params = StimParams()

# more verbose params
if(predict_emg):
    l2_weight = 1e-3
    activation_weight = 1e-7
    args.lambda_l = 1e-6
    dropout_rate = 0.5
    conn_prob = 1
    args.latent_shape = [30,30]
    lr = 0.0001
    n_epochs = 2500
else:
    l2_weight = 1e-3
    activation_weight = 1e-3
    args.lambda_l = 1e-5
    dropout_rate = 0.5
    conn_prob = 1
    args.latent_shape = [20,20]
    lr = 0.0001
    n_epochs = 2000    

<IPython.core.display.Javascript object>

In [4]:
%run ./RNN_model.ipynb

num_units = args.latent_shape[0]*args.latent_shape[1]
temp_lat = lateral_effect(args)   

input_layer = keras.Input(shape=[None,x_train.shape[-1]],batch_size=x_train.shape[0])
cell = SimpleRNN_pos_loss(num_units,activation="tanh",conn_prob = conn_prob,
                              kernel_regularizer=keras.regularizers.l2(l2_weight),activation_weight=activation_weight)
RNN_layer, state_h = keras.layers.RNN(cell,return_sequences=True,return_state=True,dynamic=True)(input_layer)                                   
dropout_layer = keras.layers.Dropout(dropout_rate)(RNN_layer)
output_layer = keras.layers.TimeDistributed(keras.layers.Dense(y_test.shape[2]))(dropout_layer)

model = tf.keras.Model(inputs=input_layer,outputs=output_layer)
stim_params.is_stim=False
optimizer = keras.optimizers.Adam(learning_rate=lr)


early_stopping_cb = keras.callbacks.EarlyStopping(patience=n_epochs/5,
                                                  restore_best_weights=True,monitor="loss")

model.compile(loss="mse",optimizer=optimizer,metrics=[keras.metrics.MeanSquaredError()])

In [ ]:
history=model.fit(x_train,y_train,epochs=n_epochs,verbose=True,callbacks=[early_stopping_cb])
layer_outputs = [layer.output for layer in model.layers]
activation_model = keras.models.Model(inputs=model.input,outputs=layer_outputs)

Train on 4 samples
Epoch 1/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1979 - mean_squared_error: 0.1939
Epoch 2/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1868 - mean_squared_error: 0.1828
Epoch 3/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1865 - mean_squared_error: 0.1825
Epoch 4/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1838 - mean_squared_error: 0.1798
Epoch 5/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1829 - mean_squared_error: 0.1789
Epoch 6/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1812 - mean_squared_error: 0.1772
Epoch 7/2500
4/4 [==============================] - 0s 88ms/sample - loss: 0.1749 - mean_squared_error: 0.1709
Epoch 8/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1775 - mean_squared_error: 0.1736
Epoch 9/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1678 - mean_squa

4/4 [==============================] - 0s 90ms/sample - loss: 0.1173 - mean_squared_error: 0.1134
Epoch 75/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.1148 - mean_squared_error: 0.1109
Epoch 76/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1134 - mean_squared_error: 0.1095
Epoch 77/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.1195 - mean_squared_error: 0.1156
Epoch 78/2500
4/4 [==============================] - 0s 99ms/sample - loss: 0.1210 - mean_squared_error: 0.1171
Epoch 79/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1147 - mean_squared_error: 0.1108
Epoch 80/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.1196 - mean_squared_error: 0.1158
Epoch 81/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.1161 - mean_squared_error: 0.1122
Epoch 82/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.1167 - mean_squared_error: 0.1128
Epoch 

Epoch 147/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0995 - mean_squared_error: 0.0957
Epoch 148/2500
4/4 [==============================] - 0s 86ms/sample - loss: 0.0963 - mean_squared_error: 0.0925
Epoch 149/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.0986 - mean_squared_error: 0.0948
Epoch 150/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0957 - mean_squared_error: 0.0919
Epoch 151/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.0952 - mean_squared_error: 0.0914
Epoch 152/2500
4/4 [==============================] - 0s 87ms/sample - loss: 0.0966 - mean_squared_error: 0.0928
Epoch 153/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0938 - mean_squared_error: 0.0900
Epoch 154/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0979 - mean_squared_error: 0.0941
Epoch 155/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0954 - mean_squar

Epoch 220/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0849 - mean_squared_error: 0.0811
Epoch 221/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0821 - mean_squared_error: 0.0783
Epoch 222/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0854 - mean_squared_error: 0.0816
Epoch 223/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0918 - mean_squared_error: 0.0880
Epoch 224/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0851 - mean_squared_error: 0.0814
Epoch 225/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0853 - mean_squared_error: 0.0815
Epoch 226/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0876 - mean_squared_error: 0.0838
Epoch 227/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0884 - mean_squared_error: 0.0846
Epoch 228/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.0825 - mean_squar

Epoch 293/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0750 - mean_squared_error: 0.0712
Epoch 294/2500
4/4 [==============================] - 0s 88ms/sample - loss: 0.0779 - mean_squared_error: 0.0741
Epoch 295/2500
4/4 [==============================] - 0s 87ms/sample - loss: 0.0789 - mean_squared_error: 0.0751
Epoch 296/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0756 - mean_squared_error: 0.0719
Epoch 297/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0755 - mean_squared_error: 0.0718
Epoch 298/2500
4/4 [==============================] - 0s 87ms/sample - loss: 0.0740 - mean_squared_error: 0.0703
Epoch 299/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0749 - mean_squared_error: 0.0711
Epoch 300/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0753 - mean_squared_error: 0.0716
Epoch 301/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0750 - mean_squar

...

Epoch 512/2500
4/4 [==============================] - 0s 88ms/sample - loss: 0.0616 - mean_squared_error: 0.0580
Epoch 513/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0572 - mean_squared_error: 0.0536
Epoch 514/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0573 - mean_squared_error: 0.0537
Epoch 515/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0606 - mean_squared_error: 0.0570
Epoch 516/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0580 - mean_squared_error: 0.0544
Epoch 517/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0597 - mean_squared_error: 0.0561
Epoch 518/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0647 - mean_squared_error: 0.0611
Epoch 519/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0584 - mean_squared_error: 0.0547
Epoch 520/2500
4/4 [==============================] - 0s 88ms/sample - loss: 0.0598 - mean_squar

Epoch 585/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0544 - mean_squared_error: 0.0508
Epoch 586/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0564 - mean_squared_error: 0.0528
Epoch 587/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0529 - mean_squared_error: 0.0493
Epoch 588/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0529 - mean_squared_error: 0.0494
Epoch 589/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0558 - mean_squared_error: 0.0522
Epoch 590/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0547 - mean_squared_error: 0.0511
Epoch 591/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0578 - mean_squared_error: 0.0542
Epoch 592/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0525 - mean_squared_error: 0.0489
Epoch 593/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0531 - mean_squar

Epoch 658/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0521 - mean_squared_error: 0.0486
Epoch 659/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0485 - mean_squared_error: 0.0449
Epoch 660/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0490 - mean_squared_error: 0.0455
Epoch 661/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0529 - mean_squared_error: 0.0493
Epoch 662/2500
4/4 [==============================] - 0s 103ms/sample - loss: 0.0493 - mean_squared_error: 0.0458
Epoch 663/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0499 - mean_squared_error: 0.0464
Epoch 664/2500
4/4 [==============================] - 0s 94ms/sample - loss: 0.0500 - mean_squared_error: 0.0465
Epoch 665/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0506 - mean_squared_error: 0.0470
Epoch 666/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0490 - mean_squa

Epoch 731/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0467 - mean_squared_error: 0.0431
Epoch 732/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0456 - mean_squared_error: 0.0421
Epoch 733/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0454 - mean_squared_error: 0.0419
Epoch 734/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0448 - mean_squared_error: 0.0413
Epoch 735/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0469 - mean_squared_error: 0.0434
Epoch 736/2500
4/4 [==============================] - 0s 94ms/sample - loss: 0.0472 - mean_squared_error: 0.0437
Epoch 737/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0456 - mean_squared_error: 0.0421
Epoch 738/2500
4/4 [==============================] - 0s 105ms/sample - loss: 0.0446 - mean_squared_error: 0.0411
Epoch 739/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0449 - mean_squa

Epoch 804/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0435 - mean_squared_error: 0.0401
Epoch 805/2500
4/4 [==============================] - 0s 92ms/sample - loss: 0.0433 - mean_squared_error: 0.0398
Epoch 806/2500
4/4 [==============================] - 0s 104ms/sample - loss: 0.0431 - mean_squared_error: 0.0396
Epoch 807/2500
4/4 [==============================] - 0s 90ms/sample - loss: 0.0439 - mean_squared_error: 0.0405
Epoch 808/2500
4/4 [==============================] - 0s 91ms/sample - loss: 0.0421 - mean_squared_error: 0.0386
Epoch 809/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0452 - mean_squared_error: 0.0417
Epoch 810/2500
4/4 [==============================] - 0s 93ms/sample - loss: 0.0414 - mean_squared_error: 0.0380
Epoch 811/2500
4/4 [==============================] - 0s 88ms/sample - loss: 0.0428 - mean_squared_error: 0.0393
Epoch 812/2500
4/4 [==============================] - 0s 89ms/sample - loss: 0.0429 - mean_squa

In [ ]:
# summary stuff
color_list=['b','g','r','c','m','y','k','tab:purple']

def get_pds(activations, start_idx,end_idx):
    # get mean activation from start_idx to end in each direction for each neuron
    mean_activation = np.mean(activations[:,start_idx:end_idx,:],axis=1)
    pds = np.zeros((mean_activation.shape[1],))
    depths = np.zeros((mean_activation.shape[1],))
    # fit mean fr with cosine model
    t=np.array(degs)
    guess_amp = 1; guess_phase = 0; guess_mean = 0;
    for unit in range(mean_activation.shape[1]):
        fit_data = mean_activation[:,unit]
        optimize_fn = lambda x: x[0]*np.cos(t+x[1]) + x[2] - fit_data
        fit = least_squares(optimize_fn,x0=[guess_amp,guess_phase,guess_mean],bounds=([0,-np.pi,-1],[3,np.pi,1]))        
        pds[unit] = fit.x[1]
        depths[unit] = fit.x[0]
        # use theta param as pd
    return pds, depths

def make_activation_grid(activations,cell,args):
    activation_grid = np.zeros((activations.shape[0],activations.shape[1],args.latent_shape[0],args.latent_shape[1]))
    pos_lis = cell.pos.astype("int")
    for i in range(activation_grid.shape[0]):
        for j in range(activation_grid.shape[1]):
            for k in range(pos_lis.shape[0]):
                activation_grid[i,j,pos_lis[k,0],pos_lis[k,1]] = activations[i,j,k]
    
    return activation_grid

        

go_cue_offset = [10,30]
activations = activation_model.predict(x_test)
rnn_activations = activations[1]
y_pred = activations[-1]
pd_data,depth_data = get_pds(rnn_activations,go_cue_offset[0]+go_cue_idx,go_cue_offset[1]+go_cue_idx)
plt.figure()
plt.subplot(2,2,1)
plt.hist(pd_data,20);
plt.subplot(2,2,2)
plt.hist(depth_data,20);
plt.subplot(2,2,3)

pd_grid = np.zeros(args.latent_shape)
pos_lis = cell.pos.astype("int")
for i in range(pd_data.shape[0]):
    pd_grid[pos_lis[i,0],pos_lis[i,1]] = pd_data[i]
plt.imshow(pd_grid)

if(predict_emg==1):
    out_idx = [12,14,16,18]
else:
    out_idx = [0,1]
    
plt.figure()
for i_out in range(len(out_idx)):
    plt.subplot(2,2,i_out+1)
    for tgt_dir_idx in range(y_test.shape[0]):
        plt.plot(np.transpose(y_pred[tgt_dir_idx,:,out_idx[i_out]]),'--',color=color_list[tgt_dir_idx]) 
        plt.plot(np.transpose(y_test[tgt_dir_idx,:,out_idx[i_out]]),'-',color=color_list[tgt_dir_idx])
        
        
rnn_activation_grid = make_activation_grid(rnn_activations,cell,args)
idx_plot = go_cue_idx
plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(rnn_activation_grid[i,idx_plot,:,:])
    
idx_plot = go_cue_idx+15
plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(rnn_activation_grid[i,idx_plot,:,:])
    
idx_plot = go_cue_idx+30
plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(rnn_activation_grid[i,idx_plot,:,:])

In [ ]:
# compare activation map vs shuffled maps to see if there is "topography"
num_shuffles = 1000

def shuffle_map(act_map):
    act_map = np.transpose(act_map)
    act_map = np.random.permutation(act_map)
    act_map = np.transpose(act_map)
    return act_map

def get_lateral_loss(act_map):
    A = tf.matmul(tf.convert_to_tensor(act_map,dtype="float32"),cell.lateral_effect)
    # loss is -1*mean of diagonal elements
    return -1*tf.reduce_mean(tf.linalg.diag_part(A))

def evaluate_topography(activations):
    num_steps = activations.shape[1]
    shuffled_loss = np.zeros((num_steps,num_shuffles))
    act_loss = np.zeros((num_steps,))
    act_loss_perc = np.zeros_like(act_loss)
    # for each step in time
    for i_step in range(num_steps):
        # shuffle map some number of times
        activation_map = activations[:,i_step,:]
        act_loss[i_step] = get_lateral_loss(activation_map)
        
        for i_shuffle in range(num_shuffles):
            shuffled_map = shuffle_map(activation_map)
            shuffled_loss[i_step,i_shuffle] = get_lateral_loss(shuffled_map)
        # get percentile of act_loss
        act_loss_perc[i_step] = percentileofscore(shuffled_loss[i_step,:],act_loss[i_step])
        
        
    return act_loss, shuffled_loss, act_loss_perc
        
activations = activation_model.predict(x_test)
rnn_activations = activations[1]    
rnn_activations.shape
act_loss, shuffled_loss, act_loss_perc = evaluate_topography(rnn_activations)
plt.figure()
plt.subplot(2,1,1)
plt.plot(act_loss_perc)
plt.subplot(2,1,2)
plt.plot(y_pred[0,:,0])

In [ ]:
# stimulation experiments    
def mean_squared_difference(x,y):
    # x,y is a tgt x time x signal matrix, get meansquared difference between x,y over time in all tgt conditions across signals
    return np.mean((x-y)**2,axis=2)

def get_recovery_time(msd): # from end of stim
    # threshold is based on msd at end of trial
    threshold = 0.1*np.max(np.max(msd))
    # find first time under threshold post end of stim
    under_threshold = np.argwhere(msd[:,stim_params.stim_time[1]:]<=threshold)
    recov_time = np.zeros((msd.shape[0],),dtype=np.int)
    
    for tgt_idx in range(msd.shape[0]):
        is_tgt = np.nonzero(under_threshold[:,0]==tgt_idx)
        if(len(is_tgt[0])==0):
            recov_time[tgt_idx] = msd.shape[1]-stim_params.stim_time[1]+1
        else:
            recov_time[tgt_idx] = under_threshold[is_tgt[0][0],1]
    return recov_time
   
def get_stim_effect(msd):
    return np.mean(msd[:,stim_params.stim_time[0]:stim_params.stim_time[1]],axis=1)
    
def get_metrics(x,y):
    # get msd
    msd = mean_squared_difference(x,y)
    
    # get recovery time based on msd
    recov_time = get_recovery_time(msd)
    stim_effect = get_stim_effect(msd)
    return (msd,recov_time,stim_effect)


def get_stim_data():
    stim_params.is_stim=False
    cell.reset_counter()
    activations = activation_model.predict(x_test)
    rnn_activations_no_stim = activations[1]
    emg_pred_no_stim = activations[-1]

    stim_params.is_stim=True
    cell.reset_counter()
    activations = activation_model.predict(x_test)
    rnn_activations_stim = activations[1]
    emg_pred_stim = activations[-1]
    
    emg_metrics = get_metrics(emg_pred_no_stim,emg_pred_stim)
    rnn_metrics = get_metrics(rnn_activations_no_stim,rnn_activations_stim)
    #rnn_activation_grid_no_stim = make_activation_grid(rnn_activations_no_stim,cell,args)
    #rnn_activation_grid_stim = make_activation_grid(rnn_activations_stim,cell,args)
    return emg_metrics, rnn_metrics
    
def run_stim_exp():    
    emg_recov = np.zeros((stim_pos_test.shape[0],len(stim_dist_tau_test),len(stim_duration_test),x_test.shape[0]))
    rnn_recov = np.zeros_like(emg_recov)
    emg_stim_effect = np.zeros_like(emg_recov)
    rnn_stim_effect = np.zeros_like(emg_recov)
    
    for i_pos in range(stim_pos_test.shape[0]):
        print(i_pos/stim_pos_test.shape[0])
        stim_params.stim_pos = stim_pos_test[i_pos,:]
        for i_tau in range(len(stim_dist_tau_test)):
            stim_params.stim_dist_tau = stim_dist_tau_test[i_tau]
            for i_dur in range(len(stim_duration_test)):
                stim_params.stim_time = [go_cue_idx+5,go_cue_idx+5+stim_duration_test[i_dur]-1]
                
                emg_metrics, rnn_metrics = get_stim_data()
                emg_recov[i_pos,i_tau,i_dur,:] = emg_metrics[1]
                emg_stim_effect[i_pos,i_tau,i_dur,:] = emg_metrics[2]
                rnn_recov[i_pos,i_tau,i_dur,:] = rnn_metrics[1]
                rnn_stim_effect[i_pos,i_tau,i_dur,:] = rnn_metrics[2]

    
    print("done")
    return emg_recov, emg_stim_effect, rnn_recov, rnn_stim_effect

stim_params = StimParams()
# get stim conditions
x = np.linspace(0, args.latent_shape[0], 5, dtype=np.float32)
y = np.linspace(0, args.latent_shape[1], 5, dtype=np.float32)
xv, yv = np.meshgrid(x, y)
xv = np.reshape(xv, (xv.size, 1))
yv = np.reshape(yv, (yv.size, 1))
stim_pos_test = np.hstack((xv, yv))
stim_dist_tau_test = [0.5,1,2,5,10,20]
stim_duration_test = [1,2,4,8,16]

emg_recov, emg_stim_effect, rnn_recov, rnn_stim_effect=run_stim_exp()

# average over stim positions
emg_recov_mean = np.mean(np.mean(emg_recov,axis=3),axis=0)
emg_stim_effect_mean = np.mean(np.mean(emg_stim_effect,axis=3),axis=0)
rnn_recov_mean = np.mean(np.mean(rnn_recov,axis=3),axis=0)
rnn_stim_effect_mean = np.mean(np.mean(rnn_stim_effect,axis=3),axis=0)

# plot effect of amplitude and stim duration
plt.figure()
ax = plt.axes(projection='3d')
for i_tau in range(len(stim_dist_tau_test)):
    for i_dur in range(len(stim_duration_test)):
        ax.scatter3D(stim_dist_tau_test[i_tau],stim_duration_test[i_dur],rnn_stim_effect_mean[i_tau,i_dur],'.',color='k')
        
ax.set_xlabel("stim tau")
ax.set_ylabel("stim duration")
ax.set_zlabel("stim effect")

# plot recovery with different amplitudes and stim durations
plt.figure()
ax = plt.axes(projection='3d')
for i_tau in range(len(stim_dist_tau_test)):
    for i_dur in range(len(stim_duration_test)):
        ax.scatter3D(stim_dist_tau_test[i_tau],stim_duration_test[i_dur],rnn_recov_mean[i_tau,i_dur],'.',color='k')
        
ax.set_xlabel("stim tau")
ax.set_ylabel("stim duration")
ax.set_zlabel("recovery")

In [ ]:
args.noise_val = 0.0
stim_params.is_stim = True
stim_params.stim_pos = np.array([10,10])
stim_params.stim_dist_tau = 4
stim_params.stim_time = [go_cue_idx+10,go_cue_idx+15]
cell.reset_counter()
activations = activation_model.predict(x_test)
rnn_activations_stim = activations[1]
y_pred_stim = activations[-1]
rnn_activation_grid_stim = make_activation_grid(rnn_activations_stim,cell,args)

stim_params.is_stim=False
cell.reset_counter()
activations = activation_model.predict(x_test)
rnn_activations_no_stim = activations[1]
y_pred_no_stim = activations[-1]
rnn_activation_grid_no_stim = make_activation_grid(rnn_activations_no_stim,cell,args)

rnn_activation_grid_diff = rnn_activation_grid_stim - rnn_activation_grid_no_stim

# plot activation difference
plt.figure()
subplot_idx = [6,2,4,8]
for i in range(len(subplot_idx)):
    plt.subplot(3,3,subplot_idx[i])
    plt.imshow(rnn_activation_grid_diff[i,stim_params.stim_time[0],:,:])
    plt.colorbar()
 

plt.figure()
subplot_idx = [6,2,4,8]
for i in range(len(subplot_idx)):
    plt.subplot(3,3,subplot_idx[i])
    plt.imshow(rnn_activation_grid_diff[i,stim_params.stim_time[1]+1,:,:])
    plt.colorbar()

# plot prediction in both cases
plt.figure()
out_idx = 7
for tgt_dir_idx in range(y_pred_no_stim.shape[0]):
    plt.subplot(3,3,tgt_dir_idx+1)
    plt.plot(y_pred_no_stim[tgt_dir_idx,:,out_idx])
    plt.plot(y_pred_stim[tgt_dir_idx,:,out_idx])
    plt.axvline(stim_params.stim_time[0],linewidth=0.2)
    plt.axvline(stim_params.stim_time[0],linewidth=0.2)
    
# example neuron
plt.figure()
unit=37
for tgt_idx in range(rnn_activations_no_stim.shape[0]):
    plt.subplot(3,3,tgt_idx+1)
    plt.plot(np.transpose(rnn_activations_no_stim[tgt_idx,:,unit]),color=color_list[tgt_idx]);
    plt.plot(np.transpose(rnn_activations_stim[tgt_idx,:,unit]),'--',color=color_list[tgt_idx]);
    plt.axvline(stim_params.stim_time[0],linewidth=0.2)
    plt.axvline(stim_params.stim_time[1],linewidth=0.2)  

In [ ]:
def plot_pca_traces(data,tgt_idx,dim_idx,is_stim=False):
    line_type = '-'
    if(is_stim):
        line_type='--'
    plt.plot(data[tgt_idx,:,dim_idx[0]],data[tgt_idx,:,dim_idx[1]],line_type,color=color_list[tgt_idx]);
    plt.plot(data[tgt_idx,0,dim_idx[0]],data[tgt_idx,0,dim_idx[1]],'.',color=color_list[tgt_idx]);
    plt.plot(data[tgt_idx,go_cue_idx,dim_idx[0]],data[tgt_idx,go_cue_idx,dim_idx[1]],'o',color=color_list[tgt_idx])
    
# dynamical analysis (PCA? and such on activations in RNN layer)
# activations is tgt_dir x time x neuron
# truncate activations from go cue to end
rnn_no_stim_trunc = rnn_activations_no_stim[:,:,:]
rnn_stim_trunc = rnn_activations_stim[:,:,:]

rnn_activations_no_stim_flat = rnn_no_stim_trunc.reshape(-1,rnn_activations_no_stim.shape[-1])
rnn_activations_stim_flat = rnn_stim_trunc.reshape(-1,rnn_activations_stim.shape[-1])

pca = PCA(n_components=10)
pca.fit(rnn_activations_no_stim_flat)

rnn_pca_no_stim = pca.transform(rnn_activations_no_stim_flat)
rnn_pca_stim = pca.transform(rnn_activations_stim_flat)
rnn_pca_no_stim = rnn_pca_no_stim.reshape(len(degs),int(rnn_pca_no_stim.shape[0]/len(degs)),rnn_pca_no_stim.shape[1])
rnn_pca_stim = rnn_pca_stim.reshape(len(degs),int(rnn_pca_stim.shape[0]/len(degs)),rnn_pca_stim.shape[1])

dims = [1,2]
plt.figure()
for tgt_idx in range(rnn_pca_no_stim.shape[0]):
    plot_pca_traces(rnn_pca_no_stim,tgt_idx,dims)
    
    
plt.figure()
for tgt_idx in range(rnn_pca_no_stim.shape[0]):
    plt.subplot(3,3,tgt_idx+1)
    plot_pca_traces(rnn_pca_no_stim,tgt_idx,dims)
    plot_pca_traces(rnn_pca_stim,tgt_idx,dims,is_stim=True)

In [ ]:
# dpca
# X = multi dimensional array. [n,t,s,d] where n is neurons, t is time, s is stimulus, d is decision
# labels = optional; list of characters to describe parameter axes ('tsd')
# n_components = Dictionary or integer; if integer use the same number of components in each marginalization,
    #otherwise every (key,value) pair refers to the number of components (value) in a marginalization (key).

def plot_dpca_traces(data,is_stim,label,dim):
    if(is_stim):
        linestyle = '--'
    else:
        linestyle = '-'
    x_data = np.arange(0,data[label].shape[1])
    
    for i_tgt in range(data[label].shape[-1]): # targets are in last dimension
        if(data[label].shape[-1] > 4):
            plt.subplot(3,3,i_tgt+1)
        else:
            plt.subplot(2,2,i_tgt+1)
        plt.plot(x_data,data[label][dim,:,i_tgt],color=color_list[i_tgt],linestyle=linestyle)    
        plt.axvline(stim_params.stim_time[0],linewidth=0.2)
        plt.axvline(stim_params.stim_time[1],linewidth=0.2)
        
X_no_stim = np.transpose(rnn_activations_no_stim,(2,1,0))
X_stim = np.transpose(rnn_activations_stim,(2,1,0))

labels = 'ts'
dpca = dPCA.dPCA(labels)
dpca.protect = ['t']
Z_no_stim = dpca.fit_transform(X_no_stim)
Z_stim = dpca.transform(X_stim)


for i_dim in range(3):
    plt.figure()
    plot_dpca_traces(Z_no_stim,0,'t',i_dim)
    plot_dpca_traces(Z_stim,1,'t',i_dim)
     
